In [4]:
# === OOS Scenario Generator for 2025-H1 (Real Wind Speed & Real Prices) ===
import numpy as np
import pandas as pd
import os
import pypsa
from datetime import datetime
import json
import requests

# === CONFIG ===
OUTPUT_DIR = "dataset2025"
RESOLUTION = "10min"
FREQ_MINUTES = 10

os.makedirs(OUTPUT_DIR, exist_ok=True)

# === Fixed Scenario Dates ===
SCENARIO_DATES = []
year=2025
SCENARIO_DATES.append({"start": f"{year}-01-01 00:00", "end": f"{year}-06-30 23:50"})


# === Load Wind Speed CSV ===
wind_speed_df = pd.read_csv("wind_speed_2025.csv", parse_dates=["timestamp"])
wind_speed_df = wind_speed_df.set_index("timestamp").sort_index()
wind_speed_df = wind_speed_df.resample(RESOLUTION).interpolate()
wind_speed_df = wind_speed_df.rename(columns={wind_speed_df.columns[0]: "wind_speed"})

# === Real Market Prices (ONLY 2025-H1) ===
def fetch_elspot_prices(start_date, end_date, price_area="DK1"):
    url = "https://api.energidataservice.dk/dataset/Elspotprices"
    params = {
        "start": f"{start_date}",
        "end": f"{end_date}",
        "filter": json.dumps({"PriceArea": [price_area]}),
        "columns": "HourDK,SpotPriceDKK",
        "sort": "HourDK ASC",
        "timezone": "dk",
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    records = response.json().get("records", [])
    df = pd.DataFrame(records)
    if df.empty:
        return pd.DataFrame(columns=["market_price"])
    df["HourDK"] = pd.to_datetime(df["HourDK"])
    df = df.drop_duplicates("HourDK").set_index("HourDK").sort_index()
    df = df.resample(RESOLUTION).interpolate()
    return df.rename(columns={"SpotPriceDKK": "market_price"})

print("📈 Fetching real market price data (2025-H1)...")
price_2025h1 = fetch_elspot_prices("2025-01-01T00:00", "2025-06-30T23:59")

# === Wind Power Curve ===
def realistic_wind_power_curve(speed):
    cut_in, rated, cut_out = 3, 12, 25
    output = np.zeros_like(speed)
    mask_ramp = (speed >= cut_in) & (speed <= rated)
    output[mask_ramp] = (speed[mask_ramp] - cut_in) / (rated - cut_in)
    output[(speed > rated) & (speed < cut_out)] = 1
    return np.clip(output, 0, 1)

# === Network Build ===
def build_network(timestamps, wind_pu, solar_pu, hydro_pu, load_profile, carbon_tax, wind_capacity):
    network = pypsa.Network()
    network.set_snapshots(timestamps)
    for bus in ["Wind", "Solar", "Hydro", "Grid"]:
        network.add("Bus", bus, carrier="electricity")
    network.add("Generator", "Wind", bus="Wind", p_nom=wind_capacity, p_max_pu=wind_pu,
                capital_cost=1000000, marginal_cost=2 + carbon_tax)
    network.add("Generator", "Solar", bus="Solar", p_nom=1000, p_max_pu=solar_pu,
                capital_cost=600000, marginal_cost=1 + carbon_tax)
    network.add("Generator", "Hydro", bus="Hydro", p_nom=1000, p_max_pu=hydro_pu,
                capital_cost=1200000, marginal_cost=0.5)
    network.add("Load", "Grid_Load", bus="Grid", p_set=load_profile)
    network.add("Generator", "Slack", bus="Grid", p_nom=1e6, p_min_pu=-1, p_max_pu=1, marginal_cost=1e6)
    for src in ["Wind", "Solar", "Hydro"]:
        network.add("Line", f"{src}_to_Grid", bus0=src, bus1="Grid", s_nom=1575, x=0.01, r=0.01, capital_cost=0)
    for comp in ["buses", "generators", "loads", "lines"]:
        getattr(network, comp)["carrier"] = "electricity"
    return network

# === Run OOS Simulation ===
def run_oos_2025_h1():
    start_date = "2025-01-01 00:00"
    end_date   = "2025-06-30 23:50"
    timestamps = pd.date_range(start=start_date, end=end_date, freq=RESOLUTION)

    wind_speed_series = wind_2025h1.reindex(timestamps, method="nearest")["wind_speed"]
    wind_pu = realistic_wind_power_curve(wind_speed_series.values)

    # Solar profile
    solar_profile = []
    for ts in timestamps:
        hour = ts.hour + ts.minute / 60
        daylight = np.exp(-0.5 * ((hour - 13) / 3) ** 2) if 6 <= hour <= 20 else 0
        seasonal = np.clip(np.cos((ts.dayofyear - 172) * 2 * np.pi / 365), 0.2, 1.0)
        value = daylight * seasonal * np.random.normal(1.0, 0.05)
        solar_profile.append(max(0, min(1, value)))
    solar_profile = np.array(solar_profile)

    # Hydro profile
    hydro_profile = 0.5 + 0.3 * np.sin(np.linspace(0, 5 * np.pi, len(timestamps))) + np.random.uniform(-0.2, 0.2)
    hydro_profile += np.random.normal(0, 0.05, len(timestamps))
    hydro_profile = np.clip(hydro_profile, 0.1, 1.0)

    # Load profile
    load_profile = 3000 + 1000 * np.sin(np.linspace(0, len(timestamps) * 2 * np.pi / (24 * 6), len(timestamps)))
    load_profile += np.random.normal(0, 200, len(timestamps))
    load_profile = np.clip(load_profile, 1000, 6000)

    carbon_tax = np.random.choice([0, 20, 50])
    wind_capacity = 100 * 15

    network = build_network(timestamps, wind_pu, solar_profile, hydro_profile, load_profile, carbon_tax, wind_capacity)
    network.optimize(solver_name="glpk")

    prices = price_2025h1.reindex(timestamps, method="nearest")["market_price"].values

    df = pd.DataFrame({
        "timestamp": timestamps,
        "wind": network.generators_t.p["Wind"].values,
        "solar": network.generators_t.p["Solar"].values,
        "hydro": network.generators_t.p["Hydro"].values,
        "load": load_profile,
        "price": prices,
        "scenario": "scenario_2025_H1_OOS_REAL"
    })

    df["revenue"] = (df["wind"] + df["solar"] + df["hydro"]) * df["price"]

    # Battery Simulation
    battery_capacity_mwh = 10.0
    battery_power_limit_mw = 5.0
    battery_efficiency = 0.9
    battery_energy = 0.0
    battery_energy_series = []
    avg_price = df["price"].mean()

    for _, row in df.iterrows():
        surplus = max(0.0, row["wind"] + row["solar"] + row["hydro"] - row["load"])
        can_charge = min(battery_power_limit_mw, surplus)
        charge_energy = can_charge * battery_efficiency

        if row["price"] < avg_price:
            battery_energy += charge_energy
        else:
            discharge = min(battery_power_limit_mw, battery_energy)
            battery_energy -= discharge

        battery_energy = max(0.0, min(battery_capacity_mwh, battery_energy))
        battery_energy_series.append(battery_energy)

    df["battery_energy"] = battery_energy_series
    df["npv"] = df["revenue"].sum() - (wind_capacity * 1000 + 600000 + 1200000)
    df["risk"] = df["revenue"].std() / max(df["revenue"].mean(), 1e-8)
    df["profit_label"] = (df["npv"] > 0).astype(int)

    output_path = f"{OUTPUT_DIR}/dataset_eval_2025_H1.csv"
    df.to_csv(output_path, index=False)
    print(f"✅ OOS dataset saved to {output_path}")

if __name__ == "__main__":
    run_oos_2025_h1()


📈 Fetching real market price data (2025-H1)...


SSLError: HTTPSConnectionPool(host='api.energidataservice.dk', port=443): Max retries exceeded with url: /dataset/Elspotprices?start=2025-01-01T00%3A00&end=2025-06-30T23%3A59&filter=%7B%22PriceArea%22%3A+%5B%22DK1%22%5D%7D&columns=HourDK%2CSpotPriceDKK&sort=HourDK+ASC&timezone=dk (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))